# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Data

In [2]:
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Kernel

In [3]:
def gaussian_kernel(X1, X2, sigma=1.0):
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return np.exp(-0.5 / sigma**2 * sqdist)

# Implementation

https://krasserm.github.io/2018/03/19/gaussian-processes/

In [4]:
# noise free

class GaussianProcess:
    def __init__(self, kernel=gaussian_kernel, alpha=1.0):
        self.alpha = alpha
        self.mu = None
        self.sigma = None
        self.kernel = kernel
        
    def fit(self, X, y):
        n, d = X.shape
        
        self.mu = np.zeros_like((n, d))
        self.sigma = np.zeros((n, n), dtype=float)

        self.sigma = self.kernel(X, X, sigma=self.alpha)
        
        self.X = X
        self.y = y
        
    def predict(self, X):
        K_s = self.kernel(self.X, X, sigma=self.alpha) 
        K_ss = self.kernel(X, X, sigma=self.alpha)
        K_inv = np.linalg.inv(self.sigma)

        mu_s = K_s.T.dot(K_inv).dot(self.y)
        sig_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
        
        return mu_s, sig_s

# Test

In [5]:
# scaling X
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Implemented gaussian process
model = GaussianProcess(kernel=gaussian_kernel)
model.fit(X_train, y_train)
mu_md , sig_md = model.predict(X_test)

# sklearn gaussian process
gauss_kernel = 1.0 * RBF(1.0)
sk_model = GaussianProcessRegressor(kernel=gauss_kernel).fit(X_train , y_train)
mu_sk , sig_sk = sk_model.predict(X_test , return_cov = True)

# comparison
mu_error = rel_error(mu_md , mu_sk)
print(f'mean error : {mu_error}')
cov_error = rel_error(sig_md , sig_sk)
print(f'var error : {cov_error}')

mean error : 6.237614473936079e-10
var error : 2.920561612801116e-06
